In [2]:
from __future__ import division, print_function, absolute_import

from skimage import color, io
from scipy.misc import imresize
import numpy as np
from sklearn.cross_validation import train_test_split
import os
from glob import glob

import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
from tflearn.metrics import Accuracy

/opt/anaconda/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
files_path = '/home/hfan/projects/Heidi/sub_train_ant/'

yes_files_path = os.path.join(files_path, 'yes_*.png')
no_files_path = os.path.join(files_path, 'no_*.png')

yes_files = sorted(glob(yes_files_path))
no_files = sorted(glob(no_files_path))

n_files = len(yes_files) + len(no_files)
print(n_files)


3672


In [4]:
print(len(yes_files),len(no_files))

1126 2546


In [3]:
size_image = 64
allX = np.zeros((n_files, size_image, size_image, 3), dtype='float64')
ally = np.zeros(n_files)
count = 0
for f in yes_files:
    try:
        img = io.imread(f)
        new_img = imresize(img, (size_image, size_image, 3))
        allX[count] = np.array(new_img)
        ally[count] = 0
        count += 1
    except:
        continue
        
for f in no_files:
    try:
        img = io.imread(f)
        new_img = imresize(img, (size_image, size_image, 3))
        allX[count] = np.array(new_img)
        ally[count] = 1
        count += 1
    except:
        continue

In [4]:
# test-train split   
X, X_test, Y, Y_test = train_test_split(allX, ally, test_size=0.1, random_state=42)

# encode the Ys
Y = to_categorical(Y, 2)
Y_test = to_categorical(Y_test, 2)

In [5]:
# normalisation of images
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Create extra synthetic training data by flipping & rotating images
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

In [6]:
# Input is a 32x32 image with 3 color channels (red, green and blue)
network = input_data(shape=[None, 64, 64, 3],
                     data_preprocessing=img_prep, 
                     data_augmentation=img_aug)

# 1: Convolution layer with 32 filters, each 3x3x3
conv_1 = conv_2d(network, 32, 3, activation='relu', name='conv_1')

# 2: Max pooling layer
network = max_pool_2d(conv_1, 2)

# 3: Convolution layer with 64 filters
conv_2 = conv_2d(network, 64, 3, activation='relu', name='conv_2')

# 4: Convolution layer with 64 filters
conv_3 = conv_2d(conv_2, 64, 3, activation='relu', name='conv_3')

# 5: Max pooling layer
network = max_pool_2d(conv_3, 2)

# 6: Fully-connected 512 node layer
network = fully_connected(network, 512, activation='relu')

# 7: Dropout layer to combat overfitting
network = dropout(network, 0.5)

# 8: Fully-connected layer with two outputs
network = fully_connected(network, 2, activation='softmax')


In [7]:
# Configure how the network will be trained
acc = Accuracy(name="Accuracy")
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.0005, metric=acc)

# Wrap the network in a model object
model = tflearn.DNN(network, checkpoint_path='model_cat_dog_6.tflearn', max_checkpoints = 3,
                    tensorboard_verbose = 3, tensorboard_dir='tmp/tflearn_logs/')


In [8]:
###################################
# Train model for 10 epochs
###################################
model.fit(X, Y, validation_set=(X_test, Y_test), batch_size=50,
      n_epoch=10, run_id='model_cat_dog_6', show_metric=True)

model.save('model_sub_ant_20170809_batch50_epoch10.tflearn')

Training Step: 669  | total loss: 0.38120 | time: 50.625s
| Adam | epoch: 010 | loss: 0.38120 - Accuracy: 0.8312 -- iter: 3300/3304
Training Step: 670  | total loss: 0.37407 | time: 52.403s
| Adam | epoch: 010 | loss: 0.37407 - Accuracy: 0.8361 | val_loss: 0.41740 - val_acc: 0.8234 -- iter: 3304/3304
--


In [10]:
model.save('model_sub_ant_20170809_batch50_epoch10.tflearn')

### Prediction

In [19]:
file_path = '/home/hfan/projects/Heidi/train_ant/yes_*_ants_*_1*.png'

files = sorted(glob(file_path))

size_image = 64

predictX = np.zeros((10, size_image, size_image, 3), dtype='float64')

count = 0
while count < 10:
    for f in files:
        try:
            img = io.imread(f)
            new_img = imresize(img, (size_image, size_image, 3))
            predictX[count] = np.array(new_img)
            count += 1
        except:
            continue


In [20]:
results = model.predict(predictX)

In [21]:
results[0:10]

array([[ 0.52945817,  0.47054178],
       [ 0.66306144,  0.3369385 ],
       [ 0.46866161,  0.53133839],
       [ 0.42004302,  0.57995701],
       [ 0.57061899,  0.42938098],
       [ 0.55663311,  0.44336686],
       [ 0.22094165,  0.77905834],
       [ 0.40128914,  0.59871083],
       [ 0.66350681,  0.33649316],
       [ 0.13621238,  0.86378765]], dtype=float32)

In [22]:
files[0:10]

['/home/hfan/projects/Heidi/train_ant/yes_09MAY2016_ants_10_1.png',
 '/home/hfan/projects/Heidi/train_ant/yes_09MAY2016_ants_10_10.png',
 '/home/hfan/projects/Heidi/train_ant/yes_09MAY2016_ants_10_100.png',
 '/home/hfan/projects/Heidi/train_ant/yes_09MAY2016_ants_10_101.png',
 '/home/hfan/projects/Heidi/train_ant/yes_09MAY2016_ants_10_102.png',
 '/home/hfan/projects/Heidi/train_ant/yes_09MAY2016_ants_10_103.png',
 '/home/hfan/projects/Heidi/train_ant/yes_09MAY2016_ants_10_104.png',
 '/home/hfan/projects/Heidi/train_ant/yes_09MAY2016_ants_10_105.png',
 '/home/hfan/projects/Heidi/train_ant/yes_09MAY2016_ants_10_11.png',
 '/home/hfan/projects/Heidi/train_ant/yes_09MAY2016_ants_10_110.png']

In [23]:
print(np.argmax(results[0]))

0


In [24]:
print(np.amax(results[0]))

0.529458
